# Intersection AIS with vessel lists

Intersection of SPIRE AIS data with lists of IUU vessels and the proactive vessel registry. IUU lists come from multiple sources most comprehensively the combined_iuu_list. In addition MMSI spoofing indicator from GFW is used. Legitimate vessels are acquired from the pro-active vessel registry.

Description of MMSI spoofing from GFW:

<em>List of MMSIs that experience substantial ID spoofing

By ID spoofing, we mean two or more vessels that are using the same MMSI at the same time. 

All the messages for an MMSI are grouped into sets of tracks that are contiguous spatially and temporally.  
Each continuous track has a unique seg_id field added.  Some tracks contain invalid lan/lon (like 91, 181) and 
are put into a special 'BAD' segment. 

The test for spoofing is fairly naive - we simple compute the extent of each segment in time, add them all up, 
and compare that to the extent of time that the vessel is active.  If the segment time is longer than the 
active time, then we know that some of the segments must overlap, and this is the indication of ID spoofing.
</em>

In [1]:
import pandas as pd
import ais_query

In [2]:
vessel_lists = pd.read_csv('iuu_list_of_lists.csv')

In [3]:
cols_position = ais_query.columns_position
cols_static = ais_query.columns_static

In [4]:
vessel_lists.list_source.value_counts()

gfw_spoofing         3385
pvr_purse_seiner      571
combined_iuu_list     120
pvr_longline           45
pvr_other              35
IATTC                   4
ICCAT                   3
Name: list_source, dtype: int64

In [5]:
sql_statement = """
SELECT imo, mmsi, name, ship_and_cargo_type, length, width, eta_date, destination
FROM ais_messages.full_year_static;
"""
ais_static = ais_query.connect(sql_statement)
ais_static.columns = ['imo', 'mmsi', 'name', 'ship_and_cargo_type', 'length', 'width', 'eta_date', 'destination']

In [6]:
iuu_vessel_list = vessel_lists[vessel_lists.IUU=='yes']
legitimate_vessel_list = vessel_lists[vessel_lists.IUU=='no']

In [7]:
iuu_imo_filter = iuu_vessel_list[iuu_vessel_list.id_type=='IMO']
iuu_mmsi_filter = iuu_vessel_list[iuu_vessel_list.id_type=='mmsi']
legitimate_imo_filter = legitimate_vessel_list[legitimate_vessel_list.id_type=='IMO']
legitimate_mmsi_filter = legitimate_vessel_list[legitimate_vessel_list.id_type=='mmsi']

## iuu vessels intersection 

In [8]:
print("unique_imo = ", len(pd.merge(ais_static, iuu_imo_filter, how='inner', left_on=['imo'], right_on=['id']).drop_duplicates().mmsi.unique()))
pd.merge(ais_static, iuu_imo_filter, how='inner', left_on=['imo'], right_on=['id']).drop_duplicates().head()

('unique_imo = ', 23)


,imo,mmsi,name,ship_and_cargo_type,length,width,eta_date,destination,IUU,id,id_type,list_source
0,9.00142e+06,440645000,TONINA 5,30.0,56.0,9.0,2016-09-16 07:40:00 UTC,BUSAN KOREA,yes,9001423,IMO,combined_iuu_list
48,9.00142e+06,440645000,TONINA 5,30.0,56.0,9.0,2017-09-16 07:40:00 UTC,BUSAN KOREA,yes,9001423,IMO,combined_iuu_list
121,9.00142e+06,306427272,TONMNA,30.0,214.0,9.0,2017-09-16 05:32:00 UTC,BUSAO KOREP,yes,9001423,IMO,combined_iuu_list
137,9.00142e+06,440643720,TONINA05 $,30.0,56.0,9.0,2017-09-16 07:40:00 UTC,BWQ)N KOREA,yes,9001423,IMO,combined_iuu_list
220,9.00142e+06,440646024,TONINA 5,30.0,56.0,9.0,2017-09-16 07:40:00 UTC,"""USAN KORAA",yes,9001423,IMO,combined_iuu_list


## iuu vessels with mmsi spoofing detected by GFW

In [9]:
print("unique_mmsi = ", len(pd.merge(ais_static, iuu_mmsi_filter, how='inner', left_on=['mmsi'], right_on=['id']).drop_duplicates().mmsi.unique()))
pd.merge(ais_static, iuu_mmsi_filter, how='inner', left_on=['mmsi'], right_on=['id']).drop_duplicates().head()

('unique_mmsi = ', 1139)


,imo,mmsi,name,ship_and_cargo_type,length,width,eta_date,destination,IUU,id,id_type,list_source
0,9148439.0,503000370,EBENEZER,70.0,42.0,8.0,2016-09-10 08:00:00 UTC,KETTERING,yes,503000370,mmsi,gfw_spoofing
175,9148439.0,503000370,EBENEZER,70.0,42.0,8.0,2017-09-10 08:00:00 UTC,KETTERING,yes,503000370,mmsi,gfw_spoofing
178,NaN,503000370,None,NaN,NaN,NaN,None,None,yes,503000370,mmsi,gfw_spoofing
310,9148439.0,503000370,EBENEZER,70.0,10.0,36.0,2017-08-06 08:00:00 UTC,KO,yes,503000370,mmsi,gfw_spoofing
327,9148439.0,503000370,EBENEZER $,70.0,42.0,8.0,2017-09-10 08:00:00 UTC,KEDTERING,yes,503000370,mmsi,gfw_spoofing


## legitimate vessels

In [11]:
print("unique_imo = ",len(pd.merge(ais_static, legitimate_imo_filter, how='inner', left_on=['imo'], right_on=['id']).drop_duplicates().mmsi.unique()))
pd.merge(ais_static, legitimate_imo_filter, how='inner', left_on=['imo'], right_on=['id']).drop_duplicates().head()

('unique_imo = ', 395)


,imo,mmsi,name,ship_and_cargo_type,length,width,eta_date,destination,IUU,id,id_type,list_source
0,8.80696e+06,224531000,PLAYA DE NOJA,30.0,76.0,14.0,None,None,no,8806955,IMO,pvr_purse_seiner
21,8.80696e+06,224531000,PLAYA DE NOJA,30.0,76.0,14.0,None,,no,8806955,IMO,pvr_purse_seiner
82,8.99628e+06,412439607,XIANG FA 8,90.0,72.0,12.0,None,None,no,8996279,IMO,pvr_purse_seiner
141,8.99628e+06,412439607,XIANG FA 8,90.0,72.0,12.0,None,,no,8996279,IMO,pvr_purse_seiner
209,8.99628e+06,412431415,XI,90.0,72.0,12.0,None,,no,8996279,IMO,pvr_purse_seiner


In [12]:
print("unique_mmsi = ",len(pd.merge(ais_static, legitimate_mmsi_filter, how='inner', left_on=['mmsi'], right_on=['id']).drop_duplicates().mmsi.unique()))
pd.merge(ais_static, legitimate_mmsi_filter, how='inner', left_on=['mmsi'], right_on=['id']).drop_duplicates().head()

('unique_mmsi = ', 0)


,imo,mmsi,name,ship_and_cargo_type,length,width,eta_date,destination,IUU,id,id_type,list_source
